# Understanding Word Embeddings for Neural Machine Translation – Fundamentals

In this notebook, we'll take a look at vectorizing words for use in machine learning models (such as neural machine translation systems) using *word2vec*. As you know, neural machine translation systems cannot process natural language directly, but need to convert it into numeric vector representations first. As the name implies, word2vec takes words from a natural language vocabulary and converts them into such vectors. The word2vec method was developed by [Mikolov et al. 2013: Efficient Estimation of Word Representations in Vector Space](https://arxiv.org/abs/1301.3781).

## 0 Housekeeping
First, we need to ensure that we have the [Gensim](https://radimrehurek.com/gensim/) library installed, which includes a popular implementation of word2vec. Run the following code to install Gensim or upgrade to its current version.

In [ ]:
# Upgrade to the newest version of pip and install or upgrade the gensim library (if necessary)
!pip install --upgrade pip
!pip install --upgrade gensim

In this notebook, we'll use a pretrained [GloVe](https://nlp.stanford.edu/projects/glove/) word embedding model provided in the [gensim-data](https://github.com/RaRe-Technologies/gensim-data) repository. The model we'll use is called 'glove-wiki-gigaword-100'. GloVe actually works a little bit differently than word2vec, but the GloVe models in the gensim-data repository were converted to word2vec format, so we can treat them as word2vec models here. The glove-wiki-gigaword-100 model contains about 6 billion words from Wikipedia and the Gigaword corpus (a corpus containing news articles from Associated Press, the New York Times, etc.) and about 400,000 unique words and corresponding word vectors, each of dimensionality 100. The model size is 128 MB, so importing it may take a few minutes. You can also train your own word2vec models in Gensim, but we'll cover this in another notebook. Run the code below to import the pretrained GloVe model.

In [ ]:
# Import the pretrained glove-wiki-gigaword-100 model from gensim-data and store it in 'word_embeddings'
import gensim.downloader as api

word_embeddings = api.load('glove-wiki-gigaword-100')

That's it. You're ready to go!

## 1 Exploring the word embedding model
We now have a pretrained word embedding model which we can explore. A trained word embedding model is also called an *embedding matrix*. The size of this embedding matrix is the number of unique words in the model (the vocabulary) times the number of neurons in the hidden layer of the neural network used to train the word embedding model (the *embed size*). The number of hidden units corresponds to the dimensionality of the word vectors in the model, which in our case is 100. So, the size of our embedding matrix is $400.000\;\times\;100 = 40.000.000$. In other words, our pretrained GloVe word embedding model/embedding matrix has a matrix size of 40 million. To get acquainted with the model, let's first have a look at a list of the 150 most frequent words in the model. This is what the code below generates (don't be suprised, there will be a lot of function words in the upper part of the list).

In [ ]:
# Generate a list of the 150 most frequent words in the model
word_embeddings.index_to_key[:150]

## 2 Exploring individual word vectors
We can query our word embedding model/embedding matrix like a look-up table, i.e., we can ask the model to look up a certain word and it will return the word vector for this word. This is what the code below does. The output will be an array of 100 floating point numbers, each representing one vector dimension (remember, the vectors in this model are 100-dimensional). Let's look up the word vector for 'government', which is listed as one of the 150 most frequent words in the model. Feel free to modify this code to explore vectors for other words in our model.

In [ ]:
# Display individual word vectors
word_embeddings['government']

Vector representations such as these are what a neural machine translation system actually processes when translating a sentence. You can think of our word embedding model/embedding matrix as a dictionary. We present the NMT system with a word such as 'government'. Since the system cannot process such natural language words, it will look up the vector of the word 'government' in the embedding matrix and use this vector representation for further processing.  
The standard transformer architecture for neural machine translation actually works with a vector dimensionality of 512, so the vector representations in these systems will be about five times bigger than the vectors in our pretrained model.

## 3 Exploring the most similar words
We can also explore semantic similarities between words. The code below gives us the ten words most similar to the word 'father' (according to what our model has learded from the underlying data). You can use any words you like in order to explore semantic similarities in the model, but this  works probably best if we use words from highly structured conceptual systems such as kinship relations.

In [ ]:
# Query the model for the words most similar to the word 'father'
word_embeddings.most_similar('father')

You may wonder how these word similarities are actually calculated and interpreted. Remember that the individual words in our model are represented by high-dimensional vectors. In order to establish the similarity between two words, we must therefore establish the similarity between the two vectors representing these words. The similarity between two vectors can be measured by calculating the *cosine similarity* between the two vectors: $$sim_{cos}(a,b) = \frac{a \cdot b}{\Vert a\Vert \Vert b\Vert}$$  
This formula means that we calculate the dot product of the two vectors and divide this by the product of their magnitudes. Regardless of the number of vector dimensions, the formula for cosine similarity will always return a value between $-1$ and $1$. A value of $-1$ means that the two vectors are exactly opposite, a value of $0$ means that the two vectors are independent (they are orthogonal or perpendicular to each other), and a value of $1$ means that the two vectors are exactly the same. As you can see in the output of the code cell above, the word listed as most similar has a value close to $1$, with the values of the words listed below becoming more and more distant from $1$. The values stated for the words in the output are the result of the cosine similarity calculation between the vectors representing these words and the vector representing the 'source word' for which we wanted to be given the most similar words.

## 4 Exploring the least similar words
Theoretically, we can also explore the dissimilarities between words (although, as you will see, the results may not make that much sense to us). The code below gives us the 10 words which, according to our model, are the least similar to the word 'father'.

In [ ]:
# Using the parameter 'negative=' reverses the most.similar() function
word_embeddings.most_similar(negative=['father'])

## 5 Calculating the similarity between two specific words
We can also calculate similarity (or dissimilarity) scores for two specific words. For example, the code below will give us a numeric similarity value for the two words 'father' and 'son' (again calculated using the cosine similarity formula above). Note that the score we get is identical to the score for 'son' in our list of the 10 words most similar to 'father' above.

In [ ]:
# Calculate semantic similarity between words using the similarity() function
word_embeddings.similarity('father', 'son')

We can also visualise the similarity or dissimilarity between two or more word vectors. For the sake of simplicity, let us assume that each dimension of a word vector corresponds to a certain semantic feature (such as *human*, *kinship*, *substance* or *action*). Mind that this is actually an oversimplification, but we accept this for didactic purposes. So, if two words share a certain semantic feature, we'd expect them to have similar values for the dimension representing this feature.  
If you run the code below, you will see a visual representation of the 100 dimensions of the similar words *father* and *son* together with the representation of the dissimilar word *government*. Create the representations and inspect the individual vector dimensions. 

In [ ]:
# Import the libraries required to visualise the word embedding vectors
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

# Create the visual representations
plt.figure(figsize=(30,3))
sns.heatmap([word_embeddings["father"], word_embeddings["son"], word_embeddings["government"]], xticklabels=True, yticklabels=True, cbar=True,
            vmin=-1, vmax=1, linewidths=0.7)
plt.show()

The first two rows (labelled *0* and *1*) are the visual representations of the 100-dimensional vectors for *father* and *son* and the third row (labelled *2*) is the representation of *government*. The values of the individual dimensions are scaled to a range between $-1$ and $1$, corresponding to the colours you see on the right. Equal colours mean equal values for the respective dimension. If you inspect the representations carefully, you will see that *father* and *son* share many dimensions with similar colour coding whereas this is not the case for the word *government*. So, the vectors of similar words will share a lot of dimensions with similar values whereas the dimensions of dissimilar words will also have dissimilar values. Note that, although we can see that a certain dimension has similar values for *father* and *son* and a dissimilar value for *government*, we cannot really tell which semantic feature is represented by this dimension. This remains the mystery of our word embedding model.  

## 6 Identifying semantic outliers
We can also present our model with a list of words and ask it to identify the word the doesn't fit in this list from a semantic point of view. In the code below, we ask the model which of the words 'father', 'mother', 'uncle' and 'car' is the odd one out.

In [ ]:
# Identify semantic outliers using the doesnt_match() function
word_embeddings.doesnt_match(['father', 'mother', 'uncle', 'car'])

## 7 Identifying analogies
We can also use our model to identify analogies such as 'queen' is to 'female' as 'king' is to '?'. In this specific example, the model identifies the correct analogy, but it breaks down when we try to identify other analogies such as 'father' is to 'son' as 'mother' is to '?'. Just modify the code accordingly to see what the model predicts in this case.

In [ ]:
# Identify top 3 analogy candidates
word_embeddings.most_similar(['queen', 'female'], ['king'], topn=3)

## 8 Visualizing the spatial arrangement word embeddings using t-SNE
For us humans, it is extremely difficult to wrap our head around the concept of *100-dimensional vectors* (although the visualization under *5* may have helped a little). In order to visualize the spatial arrangement of such vectors in vector space, we need a method which reduces the dimensionalities of these vectors to 2 or 3 while still preserving most of the semantic relations captured in the higher-dimensional vectors. A common method for dimensionality reduction is [t-SNE (t-distributed Stochastic Neighbor Embedding)](https://lvdmaaten.github.io/tsne/), which we will use here to display two-dimensional visualizations of our word embeddings. First, we need to import some additional libraries.

In [3]:
# Import the libraries required to perform t-SNE dimensionality reductions and visualizations
import numpy as np
import matplotlib.pyplot as plt
 
from sklearn.manifold import TSNE

We also need to define a function which takes our pretrained word embedding model and a specific word as arguments and then applies the t-SNE method to give us a visual representation of our word embeddings. The code used in this function was adapted from [this tutorial](https://medium.com/@aneesha/using-tsne-to-plot-a-subset-of-similar-words-from-word2vec-bb8eeaea6229).

In [4]:
# Define a function which displays our word embeddings in a two-dimensional scatter plot
def display_tsne_reduction(model, word):
    
    arr = np.empty((0,100), dtype='float')
    word_labels = [word]

    # Get the words most similar to our input word
    similar_words = model.similar_by_word(word)
    
    # Add the vector for each of these words to an array
    arr = np.append(arr, np.array([model[word]]), axis=0)
    for wrd_score in similar_words:
        wrd_vector = model[wrd_score[0]]
        word_labels.append(wrd_score[0])
        arr = np.append(arr, np.array([wrd_vector]), axis=0)
        
    # Calculate the t-SNE coordinates for 2 dimensions
    tsne = TSNE(n_components=2, random_state=0)
    np.set_printoptions(suppress=True)
    Y = tsne.fit_transform(arr)

    x_coords = Y[:, 0]
    y_coords = Y[:, 1]
    
    # Define the visiual representation of our scatter plot
    plt.scatter(x_coords, y_coords)

    for label, x, y in zip(word_labels, x_coords, y_coords):
        plt.annotate(label, xy=(x, y), xytext=(0, 0), textcoords='offset points')
    plt.xlim(x_coords.min()+0.00005, x_coords.max()+0.00005)
    plt.ylim(y_coords.min()+0.00005, y_coords.max()+0.00005)
    plt.show()

Now, we are ready to visualize semantic relations between the words in our model. The following code visualizes the 10 words with exhibited the closest proximity to the word *father* in 100-dimensional vector space. Note that these are the same words that were listed when we queried our model for the 10 words most similar to *father* above.

In [ ]:
# Display word embeddings in two-dimensional space
display_tsne_reduction(word_embeddings, 'father')

## 9 Coda: Standard word embeddings vs. contextual word embeddings

The word embeddings explored in this notebook are what you could call *normal* or *standard* word embeddings. These standard word embeddings are context-independent, meaning that each word is represented by just one vector, regardless of how many senses this word actually has and at which sentence positions it actually occurs. For example, in a sentence such as 'He left his **cell** phone in the prison **cell**', the word *cell* has two clearly distinct meanings, but standard word embedding approaches such as word2vec would collapse these different meanings into one single word vector. Contextual word embeddings are an upgraded version of these standard word embeddings, and these upgraded embeddings can be calculated by large neural language models such as Google's [BERT](https://www.aclweb.org/anthology/N19-1423/). BERT generates different embeddings for a word depending on its context, i.e., depending on the sense in which it is used and where in the sentence it occurs. In the example sentence above, BERT would create two different embeddings for the two senses of *cell*. Therefore, contextual word embeddings are more powerful representations than standard word embeddings but at the same time, creating such contextual embeddings is computationally more expensive. More information on the difference between standard and contextual word embeddings can be found [here](https://www.quora.com/What-are-the-main-differences-between-the-word-embeddings-of-ELMo-BERT-Word2vec-and-GloVe). Contextual word embeddings will be covered in a future notebook. 

## 10 Tips for further reading
This was a very brief introduction to the concept of word embeddings. Word embeddings are very powerful tools in the context of natural language processing, and we have merely scratched the surface here. If you would like to dive deeper into the topic, here are some reading tips:  
- [Alammar, Jay (2019): The Illustrated Word2vec](http://jalammar.github.io/illustrated-word2vec/)
- [Collis, Jaron (2017): Glossary of Deep Learning: Word Embeddings](https://medium.com/deeper-learning/glossary-of-deep-learning-word-embedding-f90c3cec34ca)  
- [Jedamski, Derek (2020): Advanced NLP with Python for Machine Learning](https://www.linkedin.com/learning/advanced-nlp-with-python-for-machine-learning), LinkedIn Learning course (free for students of TH Köln)
- [McCormick, Chris (2016): Word2Vec Tutorial - The Skip-Gram Model](http://mccormickml.com/2016/04/19/word2vec-tutorial-the-skip-gram-model/)  
- [McCormick, Chris (2017): Word2Vec Tutorial Part 2 - Negative Sampling](http://mccormickml.com/2017/01/11/word2vec-tutorial-part-2-negative-sampling/)
-[official gensim documentation](https://radimrehurek.com/gensim/auto_examples/core/run_core_concepts.html)